In [166]:
import time

import gizmo_analysis as gizmo
import gc_utils  # type: ignore
import h5py
import numpy as np
import pandas as pd
import utilities as ut

In [2]:
## function inputs
sim = "m12i"
# snapshot = 600
snapshot = 214  # most gcs
main_halo_tid = 25236877

# sim_dir = "../../simulations/"
sim_dir = "../../../simulations/"  # went up an extra level as in notebook folder
data_dict = {}

it_lst = [0]

fire_dir = sim_dir + sim + "/" + sim + "_res7100/"

# think about species, for basic kinematics just upload star and dark
part = gizmo.io.Read.read_snapshots(["dark", "star"], "index", snapshot, fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 214, redshift = 1.500


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  simulations/m12i/m12i_res7100/output/snapdir_214/snapshot_214.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 68214754 particles
    star      (id = 4): 2309196 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_214.0.hdf5
    snapshot_214.1.hdf5
    snapshot_214.2.hdf5
    snapshot_214.3.hdf5

* reading cosmological parameters from:  simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  simulations/m12i/m

In [3]:
proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

In [ ]:
halt = gc_utils.get_halo_tree(sim, sim_dir, assign_hosts_rotation=False)

Retrieving Halo Tree.....................: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]



# in utilities.simulation.Snapshot():
* reading:  simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 210, redshift = 1.540


# in utilities.simulation.Snapshot():
* reading:  simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 214, redshift = 1.500


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  simulations/m12i/m12i_res7100/output/snapdir_214/snapshot_214.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 68214754 particles
    star      (id = 4): 2309196 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star']

* reading particles from:
    snapshot_214.0.hdf5
    snapshot_214.1.hdf5
    snapshot_214.2.hdf5
    snapshot_214.3.hdf5

* reading cosmological parameters from:  simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle pr

In [5]:
def remove_duplicates_with_report(arr):
    arr = np.array(arr)  # Convert the list to a numpy array
    unique_elements, counts = np.unique(arr, return_counts=True)  # Get unique elements and their counts
    duplicates = unique_elements[counts > 1]  # Duplicates are those that appear more than once
    unique_list = unique_elements.tolist()  # Convert unique elements back to a list

    return unique_list, duplicates.tolist()  # Return unique and duplicate lists

In [8]:
# id_idx_map, gc_id_snap, ptype_snap = gc_utils.create_gc_part_idx_dict(part, proc_data, it, snapshot)

it = 1
it_id = gc_utils.iteration_name(it)
snap_id = gc_utils.snapshot_name(snapshot)

gc_id_snap = proc_data[it_id]["snapshots"][snap_id]["gc_id"][()]

ptype_byte_snap = proc_data[it_id]["snapshots"][snap_id]["ptype"]
ptype_snap = [ptype.decode("utf-8") for ptype in ptype_byte_snap]

# Step 1: group GCs by particle type
gc_by_ptype = {}
gc_by_ptype["star"] = []
gc_by_ptype["dark"] = []

for gc, ptype in zip(gc_id_snap, ptype_snap):
    gc_by_ptype[ptype].append(gc)

# Step 2: for each ptype, build a small dict: gc_id → index
id_idx_map = {}

for ptype, gc_ids in gc_by_ptype.items():
    ids = part[ptype]["id"]  # potentially millions of entries
    # gc_ids = np.array(gc_ids)  # small subset

    # Check which of these are in the main list
    mask = np.isin(ids, gc_ids)
    idxs = np.nonzero(mask)[0]
    found_ids = ids[idxs]

    # Build small, efficient lookup: GC ID → array index
    id_idx_map[ptype] = dict(zip(found_ids, idxs))

    # concerned abour duplciate star ids
    if ptype == "star":
        _, duplicates_ids = remove_duplicates_with_report(found_ids)

In [9]:
duplicates_ids

[42251338, 48721435]

In [45]:
i = 0
main_halo_tid = 25236877

In [58]:
ana_mask = proc_data[it_id]["source"]["analyse_flag"][()] == 1
gc_mask = proc_data[it_id]["source"]["gc_id"][()] == duplicates_ids[i]
snap_form = proc_data[it_id]["source"]["snap_zform"][ana_mask & gc_mask][0]
snap_form

210

In [61]:
# get closest public snapshot (previously calculated)
pubsnap_form = proc_data[it_id]["source"]["pubsnap_zform"][ana_mask & gc_mask][0]
pubsnap_form

214

In [62]:
group_id = np.abs(proc_data[it_id]["source"]["group_id"][ana_mask & gc_mask][0])
group_id

0

In [63]:
if group_id == 0:
    halo_tid = main_halo_tid
else:
    halo_tid = group_id

halo_tid_snap = gc_utils.get_halo_prog_at_snap(halt, halo_tid, pubsnap_form)
halo_tid_snap

6279613

In [64]:
halo_idx = np.where(halt["tid"] == halo_tid_snap)[0][0]
snapcheck = halt["snapshot"][halo_idx]
snapcheck

214

In [65]:
halo_pos = halt["position"][halo_idx]
halo_pos

array([42694.8  , 42784.387, 45005.57 ], dtype=float32)

In [73]:
part_form = gizmo.io.Read.read_snapshots(["star"], "index", pubsnap_form, fire_dir)

In [145]:
part_idxs = np.where(part["star"]["id"] == duplicates_ids[i])[0]
part_snapform = part["star"].prop("form.snapshot", part_idxs)

idx = np.where(part_snapform == snap_form)[0]
idx

array([ 5, 13, 14])

In [272]:
part_form_idxs = np.where(part_form["star"]["id"] == duplicates_ids[i])[0]
part_snapform = part_form["star"].prop("form.snapshot", part_idxs)
part_snapform

array([209, 207, 214, 205, 212, 210, 211, 214, 212, 209, 205, 209, 214,
       210, 210, 212, 211, 205, 207], dtype=int32)

In [276]:
idxs = np.where(part_snapform == snap_form)[0]
part_interest_idxs = part_form_idxs[idxs]
part_interest_idxs

array([ 524234, 1450863, 1476832])

In [274]:
part_form["star"]["position"][part_interest_idxs]

array([[42695.99932255, 42785.27086674, 45008.4481108 ],
       [42697.15656516, 42784.03937121, 45008.1257833 ],
       [42690.09579992, 42784.40084581, 45027.30340428]])

In [275]:
ut.coordinate.get_distances(
    part_form["star"]["position"][part_interest_idxs],
    halo_pos,
    part_form.info["box.length"],
    part_form.snapshot["scalefactor"],
    total_distance=True,
)  # [kpc physical]

array([1.29614456, 1.39718569, 8.89462153])

In [151]:
j = 1
x = part_form["star"]["position"][part_interest_idxs][j][0] - halo_pos[0]
y = part_form["star"]["position"][part_interest_idxs][j][1] - halo_pos[1]
z = part_form["star"]["position"][part_interest_idxs][j][2] - halo_pos[2]

r2_s = (x**2 + y**2 + z**2) * (part_form.snapshot["scalefactor"] / part_form.info["hubble"]) ** 2
r = np.sqrt(r2_s)
r

1.990293010174199

In [152]:
part_form["star"]["form.scalefactor"][part_interest_idxs]

array([0.39315334, 0.39361042, 0.3927175 ], dtype=float32)

In [163]:
part_form["star"]["id.generation"][part_interest_idxs]

array([ 8, 25,  1], dtype=uint32)

In [130]:
gas_part = gizmo.io.Read.read_snapshots(["gas"], "index", pubsnap_form, fire_dir)

In [137]:
gas_part = np.where(gas_part["gas"]["id"] == duplicates_ids[i])
gas_part

TypeError: tuple indices must be integers or slices, not str

In [165]:
part_point = gizmo.io.Read.read_snapshots(
    ["star", "gas"], "index", pubsnap_form, fire_dir, assign_pointers=True
)

In [169]:
# get public snapshot at snapshot before formation

public_snapshot_file = sim_dir + "snapshot_times_public.txt"
pub_data = pd.read_table(public_snapshot_file, comment="#", header=None, sep=r"\s+")
pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]
pub_snaps = np.array(pub_data["index"], dtype=int)

preform_snap_idx = np.where(pub_snaps == pubsnap_form)[0][0] - 1
preform_snap = pub_snaps[preform_snap_idx]
preform_snap

172

In [170]:
part_point_pre = gizmo.io.Read.read_snapshots(
    ["star", "gas"], "index", preform_snap, fire_dir, assign_pointers=True
)

In [172]:
part_point_pre.Pointer.add_intermediate_pointers(part_point.Pointer)

In [179]:
"""Parameters
    ----------
    species_name_from : str
        name of species at the reference (later, z0) snapshot
    species_names_to : str or list
        name[s] of species to get pointers to at the (earlier, z) snapshot
    part_indices : arr
        indices of particles at the reference (later, z0) snapshot
    forward : bool
        whether to get pointers from the (earlier, z) snapshot to the reference (later, z0)
        snapshot, that is, tracking forward in time default (forward=False) is tracking
        backwards in time
    intermediate_snapshot : bool
        whether to get pointers between z and an intermediate snapshot (at z > 0)
        default (intermediate_snapshot=False) is to get pointers to/from z0
    return_single_array : bool
        if tracking single species at both snapshots, whether to return single array of
        pointer indices (instead of dictionary of pointers that includes species names)
"""

pointers = part_point_pre.Pointer.get_pointers(
    species_name_from="star", species_names_to=["gas", "star"], intermediate_snapshot=True
)

In [183]:
star_indices_at_pub_form = part_interest_idxs
# star_indices_at_pub_form = np.array([0])
pointers["species"][star_indices_at_pub_form]
pointers["index"][star_indices_at_pub_form]

array([-71036560, -71036560, -71036560], dtype=int32)

In [188]:
# If you just want to track a single species (star -> star or gas -> gas) between 2 snapshots,
# read_pointers_between_snapshots() makes it easy to get the pointer indices between any 2 snapshots

ParticlePointer = gizmo.track.ParticlePointerClass(simulation_directory=fire_dir)

# tracking forward in time
pointers_pre_to_post = ParticlePointer.read_pointers_between_snapshots(
    snapshot_index_from=preform_snap, snapshot_index_to=pubsnap_form, species_name="star"
)

In [246]:
part_z0 = gizmo.io.Read.read_snapshots(["star", "gas"], "index", 600, fire_dir)

part_form = gizmo.io.Read.read_snapshots(
    ["star", "gas"], "index", pubsnap_form, fire_dir, assign_pointers=True
)
part_preform = gizmo.io.Read.read_snapshots(
    ["star", "gas"], "index", preform_snap, fire_dir, assign_pointers=True
)

In [267]:
pointers = part_form.Pointer.get_pointers(
    species_name_from="star", species_names_to=["star", "gas"], forward=True
)

# indices_at_form = part_interest_idxs
indices_at_form = np.array([0, 1])
indices_at_z0 = pointers["index"][indices_at_form]
species_at_z0 = pointers["species"][indices_at_form]
indices_at_form, indices_at_z0, species_at_z0

(array([0, 1]),
 array([4357964, 2985361], dtype=int32),
 array(['star', 'star'], dtype='<U4'))

In [271]:
part_form["star"]["id"][indices_at_form]

array([39840433, 38206582], dtype=uint32)

In [270]:
part_z0["star"]["id"][indices_at_z0]

array([39840433, 38206582], dtype=uint32)

In [ ]:
# part_2.Pointer.add_intermediate_pointers(part_1.Pointer)

In [ ]:
# pointers = part_2.Pointer.get_pointers(
#     species_name_from="star", species_names_to="gas", intermediate_snapshot=True
# )

In [ ]:
# star_indices_at_1 = part_interest_idxs

# # read from 2 to 1
# pointers

array([-69093569,  39890936, -69093569, ..., -69093569, -69093569,
       -69093569], dtype=int32)

In [207]:
np.where(pointers["species"] == "gas")[0]

array([      1,       4,       8, ..., 2309160, 2309166, 2309169])